In [1]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
import brunoflow as bf
from brunoflow.ad.utils import check_node_equals_tensor, check_node_allclose_tensor
from jax import numpy as jnp
import transformers
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForMaskedLM, 
    BertForMaskedLM, 
    BertTokenizer, 
    BertTokenizerFast, 
    BertEmbeddings,
    BfBertEmbeddings,
    BfBertEncoder,
    BertConfig,
    BfBertSelfAttention,
)

torch.manual_seed(0)

env: XLA_PYTHON_CLIENT_PREALLOCATE=false


/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Establish data
model_id = "google/bert_uncased_L-2_H-128_A-2"
config = BertConfig.from_pretrained(pretrained_model_name_or_path="../../brunoflow/models/bert/config-tiny.json")
tokenizer = BertTokenizerFast.from_pretrained(model_id)
text = ["hello I want to eat some [MASK] meat today. It's thanksgiving [MASK] all!", "yo yo what's up"]

# tokenize text and pass into model
tokens = tokenizer(text, return_tensors="pt", padding=True)
input_ids = tokens["input_ids"]
jax_input_ids = bf.Node(jnp.array(input_ids.numpy(), dtype=int), name="inputs")
print(input_ids)

2023-01-10 13:26:05.608504: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


tensor([[  101,  7592,  1045,  2215,  2000,  4521,  2070,   103,  6240,  2651,
          1012,  2009,  1005,  1055, 15060,   103,  2035,   999,   102],
        [  101, 10930, 10930,  2054,  1005,  1055,  2039,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]])


In [3]:
# Create BfBertEmbeddings and BertEmbeddings
bf_embs = BfBertEmbeddings(config)
# Visualize output of forward pass of BfBertEmbeddings
bf_embs.train(False)
out_bf = bf_embs(input_ids=jax_input_ids)
out_bf.visualize(collapse_to_modules=True)
print(bf_embs)

/home/kevin/code/rycolab/transformers/src/transformers/models/bert/modeling_bf_bert.py:178: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in zeros is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  bf.Parameter(jnp.zeros(self.position_ids.shape, dtype=jnp.int64), name="position_ids"),


BfBertEmbeddings(
  (word_embeddings): Embedding(word_embeddings)(30522, 128, padding_idx=0)
  (position_embeddings): Embedding(position_embeddings)(512, 128)
  (token_type_embeddings): Embedding(token_type_embeddings)(2, 128)
  (LayerNorm): LayerNorm(in BfBertEmbeddings)((128,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0)
)


In [4]:
# Init inputs to bf and torch models
hidden_states_torch = torch.randn(size=(2, 19, 128))
attention_mask_torch = torch.randn(size=(2, 1, 1, 19))

hidden_states = bf.Node(jnp.array(hidden_states_torch.numpy(), dtype=jnp.float64), name="hidden_states")
attention_mask = bf.Node(jnp.array(attention_mask_torch.numpy(), dtype=jnp.float64), name="attention_mask")
bert_enc = BfBertEncoder(config)
out = bert_enc(hidden_states, attention_mask)
out.last_hidden_state.visualize(collapse_to_modules=False)

/tmp/ipykernel_207089/1679073517.py:5: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  hidden_states = bf.Node(jnp.array(hidden_states_torch.numpy(), dtype=jnp.float64), name="hidden_states")
/tmp/ipykernel_207089/1679073517.py:6: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  attention_mask = bf.Node(jnp.array(attention_mask_torch.numpy(), dtype=jnp.float64), name="attention_mask")


In [7]:
# Init inputs to bf and torch models
hidden_states_torch = torch.randn(size=(2, 19, 128))
attention_mask_torch = torch.randn(size=(2, 1, 1, 19))

hidden_states = bf.Node(jnp.array(hidden_states_torch.numpy(), dtype=jnp.float64), name="hidden_states")
attention_mask = bf.Node(jnp.array(attention_mask_torch.numpy(), dtype=jnp.float64), name="attention_mask")
bert = BfBertSelfAttention(config)
out = bert(hidden_states, attention_mask)
out[0].visualize(collapse_to_modules=False)


/tmp/ipykernel_128456/2583383163.py:5: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  hidden_states = bf.Node(jnp.array(hidden_states_torch.numpy(), dtype=jnp.float64), name="hidden_states")
/tmp/ipykernel_128456/2583383163.py:6: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  attention_mask = bf.Node(jnp.array(attention_mask_torch.numpy(), dtype=jnp.float64), name="attention_mask")
